In [363]:
from logzero import logger
from SmartApi.smartConnect import SmartConnect
from sqlalchemy import create_engine
from urllib.parse import quote 
from datetime import datetime, timedelta

import pyotp
import json
import pandas as pd
import requests

api_key = 'BP42pHUk'
username = 'R60865380'
pwd = '6713'
smartApi = SmartConnect(api_key)

# List of possible stock names
# url https://www.nseindia.com/api/master-quote
stock_names = sorted([
    "AARTIIND","ABB","ABBOTINDIA","ABCAPITAL","ABFRL","ACC","ADANIENT","ADANIPORTS","ALKEM",
    "AMBUJACEM","APOLLOHOSP","APOLLOTYRE","ASHOKLEY","ASIANPAINT","ASTRAL","ATUL","AUBANK",
    "AUROPHARMA","AXISBANK","BAJAJ-AUTO","BAJAJFINSV","BAJFINANCE","BALKRISIND","BALRAMCHIN",
    "BANDHANBNK","BANKBARODA","BATAINDIA","BEL","BERGEPAINT","BHARATFORG","BHARTIARTL","BHEL",
    "BIOCON","BOSCHLTD","BPCL","BRITANNIA","BSOFT","CANBK","CANFINHOME","CHAMBLFERT","CHOLAFIN",
    "CIPLA","COALINDIA","COFORGE","COLPAL","CONCOR","COROMANDEL","CROMPTON","CUB","CUMMINSIND",
    "DABUR","DALBHARAT","DEEPAKNTR","DIVISLAB","DIXON","DLF","DRREDDY","EICHERMOT","ESCORTS",
    "EXIDEIND","FEDERALBNK","GAIL","GLENMARK","GMRINFRA","GNFC","GODREJCP","GODREJPROP",
    "GRANULES","GRASIM","GUJGASLTD","HAL","HAVELLS","HCLTECH","HDFCAMC","HDFCBANK","HDFCLIFE",
    "HEROMOTOCO","HINDALCO","HINDCOPPER","HINDPETRO","HINDUNILVR","ICICIBANK","ICICIGI",
    "ICICIPRULI","IDEA","IDFC","IDFCFIRSTB","IEX","IGL","INDHOTEL","INDIACEM","INDIAMART",
    "INDIGO","INDUSINDBK","INDUSTOWER","INFY","IOC","IPCALAB","IRCTC","ITC","JINDALSTEL",
    "JKCEMENT","JSWSTEEL","JUBLFOOD","KOTAKBANK","LALPATHLAB","LAURUSLABS","LICHSGFIN","LT",
    "LTF","LTIM","LTTS","LUPIN","M&M","M&MFIN","MANAPPURAM","MARICO","MARUTI","MCDOWELL-N",
    "MCX","METROPOLIS","MFSL","MGL","MOTHERSON","MPHASIS","MRF","MUTHOOTFIN","NATIONALUM",
    "NAUKRI","NAVINFLUOR","NESTLEIND","NMDC","NTPC","OBEROIRLTY","OFSS","ONGC","PAGEIND","PEL",
    "PERSISTENT","PETRONET","PFC","PIDILITIND","PIIND","PNB","POLYCAB","POWERGRID","PVRINOX",
    "RAMCOCEM","RBLBANK","RECLTD","RELIANCE","SAIL","SBICARD","SBILIFE","SBIN","SHREECEM",
    "SHRIRAMFIN","SIEMENS","SRF","SUNPHARMA","SUNTV","SYNGENE","TATACHEM","TATACOMM","TATACONSUM",
    "TATAMOTORS","TATAPOWER","TATASTEEL","TCS","TECHM","TITAN","TORNTPHARM","TRENT","TVSMOTOR",
    "UBL","ULTRACEMCO","UNITDSPR","UPL","VEDL","VOLTAS","WIPRO","ZYDUSLIFE"
], key=len, reverse=True)

nse_holidays_2024 = [ 
    '2024-01-26',  # Republic Day
    '2024-03-08',  # Mahashivratri
    '2024-03-25',  # Holi
    '2024-03-29',  # Good Friday
    '2024-04-11',  # Id-Ul-Fitr (Ramadan Eid)
    '2024-04-17',  # Shri Ram Navmi
    '2024-05-01',  # Maharashtra Day
    '2024-06-17',  # Bakri Id
    '2024-07-17',  # Moharram
    '2024-08-15',  # Independence Day/Parsi New Year
    '2024-10-02',  # Mahatma Gandhi Jayanti
    '2024-11-01',  # Diwali Laxmi Pujan (Muhurat Trading will be conducted)
    '2024-11-15',  # Gurunanak Jayanti
    '2024-12-25',  # Christmas
    '2024-04-14',  # Dr. Baba Saheb Ambedkar Jayanti (Sunday)
    '2024-04-21',  # Shri Mahavir Jayanti (Sunday)
    '2024-09-07',  # Ganesh Chaturthi (Saturday)
    '2024-10-12',  # Dussehra (Saturday)
    '2024-11-02',  # Diwali-Balipratipada (Saturday)
]
# year = 2024
# month = 7
# end_date = 2024-07-02

res_ticker_df = pd.DataFrame([])
res_candle_stick_df = pd.DataFrame([])

try:
    token = "ARA7CTUQBDWODFCWKVSAXTLR4U"
    totp = pyotp.TOTP(token).now()
except Exception as e:
    logger.error("Invalid Token: The provided token is not valid.")
    raise e

correlation_id = "abcde"
data = smartApi.generateSession(username, pwd, totp)

[I 240723 13:27:05 smartConnect:121] in pool


In [364]:
if data['status'] == False:
    logger.error(data)
else:
    # logger.info(f"data: {data}")
    authToken = data['data']['jwtToken']
    refreshToken = data['data']['refreshToken']
    feedToken = smartApi.getfeedToken()
    # logger.info(f"Feed-Token :{feedToken}")
    res = smartApi.getProfile(refreshToken)
    # logger.info(f"Get Profile: {res}")
    smartApi.generateToken(refreshToken)
    res=res['data']['exchanges']
    #search for symbol
    # logger.info(f"Search Scrip Data : {searchScripData}")


In [365]:
def getInstrumentData(exchange,searchscrip):
    searchScripData = smartApi.searchScrip(exchange, searchscrip)
    # print(exchange,searchscrip)
    return searchScripData

In [366]:
def getResponse(url):
    headers = {
        "user-agent": "Chrome/80.0.3987.149 Safari/537.36",
        "accept-language": "en,gu;q=0.9,hi;q=0.8",
        "Accept": "application/json"
    }
    response = requests.get(url, headers=headers, timeout=5)
    return response

In [367]:
def getValidInstrumentTickData(symbolToken, interval='1minute', fromDate='2024-07-01', toDate='2024-07-01'):
    # url = 'https://api.upstox.com/v2/historical-candle/NSE_FO|134606/1minute/2024-07-11/2024-07-1'
    uplinkURL = f'https://api.upstox.com/v2/historical-candle/NSE_FO|{symbolToken}/{interval}/{toDate}/{fromDate}'
    # print(uplinkURL)
    response = getResponse(uplinkURL)
    # print(response.text)
    if response.ok:
        # print(uplinkURL)
        df = pd.DataFrame(response.json()['data']['candles'], columns=['time_stamp','open','high','low','close','volume','open_interest'])
        if not df.empty:
            df.insert(1, 'ticker_id', symbolToken+'-'+fromDate)
            df['cs_id'] = df[['ticker_id', 'time_stamp']].agg('-'.join, axis=1)
            return df
        # print (df)
        # return df
        # print(symbolToken)
        # return symbolToken
    # else:
    #     print(f'no{symbolToken}')
        # return None

In [368]:
def generate_dates(year, month, holidays, end_date):
    # Get the first and last day of the month
    start_date = datetime(year, month, 1)
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    # if month == 12:
    #     end_date = datetime(year + 1, 1, 1) - timedelta(days=1)
    # else:
    #     end_date = datetime(year, month + 1, 1) - timedelta(days=1)

    # Generate all dates in the month
    all_dates = pd.date_range(start=start_date, end=end_date).tolist()

    # Filter out weekends and holidays
    valid_dates = [date for date in all_dates if date.weekday() < 5 and date.strftime('%Y-%m-%d') not in holidays]

    # Format the dates as 'yyyy-mm-dd'
    formatted_dates = [date.strftime('%Y-%m-%d') for date in valid_dates]

    return formatted_dates

In [369]:
# Define a function to extract the components
def extract_components(s):
    parts = s.split('-')
    tradedate = parts[0] + '-' + parts[1] + '-' + parts[2]
    option_details = parts[3]
    
    # Identify stock name
    stockname = next((name for name in stock_names if option_details.startswith(name)), None)
    
    if stockname:
        len_stockname = len(stockname)
    else:
        len_stockname = 0

    # Extract expiry date in the format dd-MMM-yy
    expiry_date_raw = option_details[len_stockname:len_stockname+7]  # '25JUL24'
    expirydate = expiry_date_raw[:2] + '-' + expiry_date_raw[2:5].upper() + '-' + '20' + expiry_date_raw[5:]
    
    # Extract strike price
    strikeprice = ''.join(filter(str.isdigit, option_details[len_stockname+7:]))
    
    # Extract option type
    optiontype = ''.join(filter(str.isalpha, option_details[len_stockname+7+len(strikeprice):]))
    
    return pd.Series([tradedate, stockname, expirydate, strikeprice, optiontype])

In [370]:
# fnoStocksURL = 'https://www.nseindia.com/api/master-quote'
# response = getResponse(fnoStocksURL)
# print(response.json())

for date in generate_dates(2024, 7, nse_holidays_2024, '2024-07-22'):
    for stock in stock_names:
        print (f'Dumping {stock} on {date}')
        instrumentData = getInstrumentData('NFO', stock)
        # print(instrumentData)
        df = pd.DataFrame(instrumentData['data'], columns=['exchange','tradingsymbol','symboltoken'])
        curr_mon_df = df[df['tradingsymbol'].str.contains("JUL") & ~df['tradingsymbol'].str.contains("FUT")]
        # print(curr_mon_df)
        for symbolToken in curr_mon_df['symboltoken']:
            valid_df = getValidInstrumentTickData(symbolToken, '1minute', date, date)
            res_candle_stick_df = pd.concat([res_candle_stick_df, valid_df])
        # print(curr_mon_df)
        curr_mon_df['tradingsymbol'] = curr_mon_df['tradingsymbol'].apply(lambda x: date+'-'+x)
        # curr_mon_df = curr_mon_df.set_index('tradingsymbol')
        # curr_mon_df.insert(loc=0, column='timeTradingSymbol', value=time_trading_symbol)
        # print(curr_mon_df)
        res_ticker_df = pd.concat([res_ticker_df, curr_mon_df])
        print (f'Done {stock} on {date}')
# print(curr_mon_df.size)
# print(curr_mon_df.iloc[2]['valid_df'])
# print(res_ticker_df)

Dumping ABBOTINDIA on 2024-07-01


[I 240723 13:27:06 smartConnect:485] Search successful. Found 440 trading symbols for the given query:
    1. exchange: NFO, tradingsymbol: ABBOTINDIA25JUL2415500CE, symboltoken: 76572
    2. exchange: NFO, tradingsymbol: ABBOTINDIA25JUL2415500PE, symboltoken: 76573
    3. exchange: NFO, tradingsymbol: ABBOTINDIA25JUL2416000CE, symboltoken: 67235
    4. exchange: NFO, tradingsymbol: ABBOTINDIA25JUL2416000PE, symboltoken: 67236
    5. exchange: NFO, tradingsymbol: ABBOTINDIA25JUL2416500CE, symboltoken: 49922
    6. exchange: NFO, tradingsymbol: ABBOTINDIA25JUL2416500PE, symboltoken: 49923
    7. exchange: NFO, tradingsymbol: ABBOTINDIA25JUL2417000CE, symboltoken: 67239
    8. exchange: NFO, tradingsymbol: ABBOTINDIA25JUL2417000PE, symboltoken: 67240
    9. exchange: NFO, tradingsymbol: ABBOTINDIA25JUL2417500CE, symboltoken: 67241
    10. exchange: NFO, tradingsymbol: ABBOTINDIA25JUL2417500PE, symboltoken: 67242
    11. exchange: NFO, tradingsymbol: ABBOTINDIA25JUL2418000CE, symboltoken:

Done ABBOTINDIA on 2024-07-01
Dumping ADANIPORTS on 2024-07-01


[I 240723 13:27:31 smartConnect:485] Search successful. Found 280 trading symbols for the given query:
    1. exchange: NFO, tradingsymbol: ADANIPORTS25JUL241000CE, symboltoken: 68216
    2. exchange: NFO, tradingsymbol: ADANIPORTS25JUL241000PE, symboltoken: 68217
    3. exchange: NFO, tradingsymbol: ADANIPORTS25JUL241020PE, symboltoken: 57768
    4. exchange: NFO, tradingsymbol: ADANIPORTS25JUL241040CE, symboltoken: 68224
    5. exchange: NFO, tradingsymbol: ADANIPORTS25JUL241040PE, symboltoken: 68225
    6. exchange: NFO, tradingsymbol: ADANIPORTS25JUL241060CE, symboltoken: 57769
    7. exchange: NFO, tradingsymbol: ADANIPORTS25JUL241060PE, symboltoken: 57770
    8. exchange: NFO, tradingsymbol: ADANIPORTS25JUL241080CE, symboltoken: 68301
    9. exchange: NFO, tradingsymbol: ADANIPORTS25JUL241080PE, symboltoken: 68302
    10. exchange: NFO, tradingsymbol: ADANIPORTS25JUL241100CE, symboltoken: 79840
    11. exchange: NFO, tradingsymbol: ADANIPORTS25JUL241100PE, symboltoken: 79841
    

Done ADANIPORTS on 2024-07-01
Dumping APOLLOHOSP on 2024-07-01


...

[I 240723 13:41:21 smartConnect:485] Search successful. Found 340 trading symbols for the given query:
    1. exchange: NFO, tradingsymbol: MANAPPURAM25JUL24115CE, symboltoken: 49773
    2. exchange: NFO, tradingsymbol: MANAPPURAM25JUL24115PE, symboltoken: 49774
    3. exchange: NFO, tradingsymbol: MANAPPURAM25JUL24120CE, symboltoken: 53412
    4. exchange: NFO, tradingsymbol: MANAPPURAM25JUL24120PE, symboltoken: 53413
    5. exchange: NFO, tradingsymbol: MANAPPURAM25JUL24125CE, symboltoken: 57308
    6. exchange: NFO, tradingsymbol: MANAPPURAM25JUL24125PE, symboltoken: 57309
    7. exchange: NFO, tradingsymbol: MANAPPURAM25JUL24130CE, symboltoken: 49466
    8. exchange: NFO, tradingsymbol: MANAPPURAM25JUL24130PE, symboltoken: 49467
    9. exchange: NFO, tradingsymbol: MANAPPURAM25JUL24135CE, symboltoken: 53414
    10. exchange: NFO, tradingsymbol: MANAPPURAM25JUL24135PE, symboltoken: 53415
    11. exchange: NFO, tradingsymbol: MANAPPURAM25JUL24140CE, symboltoken: 127090
    12. exchan

Done MANAPPURAM on 2024-07-01
Dumping MCDOWELL-N on 2024-07-01


[I 240723 13:41:51 smartConnect:488] Search successful. No matching trading symbols found for the given query.


Done MCDOWELL-N on 2024-07-01
Dumping METROPOLIS on 2024-07-01


DataException: Couldn't parse the JSON response received from the server: b'Access denied because of exceeding access rate'

In [391]:
# print(pd.read_json(res_ticker_df.head(1).iloc[0]['valid_df']))
# print(res_candle_stick_df)
print(res_ticker_df)
print(res_candle_stick_df)

    exchange                        tradingsymbol symboltoken
0        NFO  2024-07-01-ABBOTINDIA25JUL2415500CE       76572
1        NFO  2024-07-01-ABBOTINDIA25JUL2415500PE       76573
2        NFO  2024-07-01-ABBOTINDIA25JUL2416000CE       67235
3        NFO  2024-07-01-ABBOTINDIA25JUL2416000PE       67236
4        NFO  2024-07-01-ABBOTINDIA25JUL2416500CE       49922
..       ...                                  ...         ...
123      NFO    2024-07-01-MANAPPURAM25JUL24310PE       57001
124      NFO    2024-07-01-MANAPPURAM25JUL24315CE       57002
125      NFO    2024-07-01-MANAPPURAM25JUL24315PE       57003
126      NFO    2024-07-01-MANAPPURAM25JUL24320CE       47031
127      NFO    2024-07-01-MANAPPURAM25JUL24320PE       47032

[4523 rows x 3 columns]
                    time_stamp          ticker_id   open   high    low  close  \
0    2024-07-01T15:29:00+05:30   67272-2024-07-01  65.00  65.00  65.00  65.00   
1    2024-07-01T15:28:00+05:30   67272-2024-07-01  65.00  65.00  65.0

In [ ]:
# curr_mon_df.to_pickle('data.pkl')
# curr_mon_df[curr_mon_df.index.duplicated()]
ticker_df=pd.DataFrame(columns=['ticker_id','trade_date','stock_name','expiry_date','strike_price','option_type'])
# print(ticker_df)
ticker_df['ticker_id'] = res_ticker_df ['symboltoken']
ticker_df[['trade_date','stock_name','expiry_date','strike_price','option_type']] = res_ticker_df['tradingsymbol'].apply(extract_components)
ticker_df['ticker_id'] = ticker_df[['ticker_id', 'trade_date']].agg('-'.join, axis=1)
# print(ticker_df)


In [ ]:
# DB ACTIVITY

# Create a PostgreSQL engine
engine = create_engine('postgresql+psycopg2://sd_admin:%s@192.168.1.72:5430/stock-dumps'% quote('sdadmin@postgres'))
print(engine)
# Save the DataFrame to a table named 'optstkdata'
# curr_mon_df.to_sql('optstkdata',if_exists='append', con=engine, index=True)
ticker_df.to_sql('ticker',schema='options',if_exists='append', con=engine, index=False)
res_candle_stick_df.to_sql('candle_stick',schema='options',if_exists='append', con=engine, index=False)

In [389]:
pip freeze > req.txt

Note: you may need to restart the kernel to use updated packages.


In [393]:
import sys
sys.path

['/home/cgraaaj/.asdf/installs/python/3.12.4/lib/python312.zip',
 '/home/cgraaaj/.asdf/installs/python/3.12.4/lib/python3.12',
 '/home/cgraaaj/.asdf/installs/python/3.12.4/lib/python3.12/lib-dynload',
 '',
 '/home/cgraaaj/.asdf/installs/python/3.12.4/lib/python3.12/site-packages']